In [1]:
import pandas as pd
from project_demo.tools.features_engine import *
import copy
from sklearn.pipeline import *
from keras.models import *
from keras.layers import *
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
import pandas as pd
from project_demo.tools.multi_apply import *
from project_demo.tools.evaluate import *
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
train_identity = pd.read_csv('../data/train_identity.csv')
test_identity = pd.read_csv('../data/test_identity.csv')
train_identity['split']=1
test_identity['split']=2
identity = pd.concat([train_identity,test_identity])

In [3]:
print(identity.columns)

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo', 'split'],
      dtype='object')


In [4]:
train_identity['DeviceInfo'].value_counts()  # categoricalDomain=['id_12','id_15','id_16','id_23','id_27','id_28',
                                        # 'id_29','id_30','id_31','id_33','id_34','id_35','id_36','id_37'
                                        # 'id_38','DeviceType','DeviceInfo']

Windows                           47722
iOS Device                        19782
MacOS                             12573
Trident/7.0                        7440
rv:11.0                            1901
rv:57.0                             962
SM-J700M Build/MMB29K               549
SM-G610M Build/MMB29K               461
SM-G531H Build/LMY48B               410
rv:59.0                             362
SM-G935F Build/NRD90M               334
SM-G955U Build/NRD90M               328
SM-G532M Build/MMB29T               316
ALE-L23 Build/HuaweiALE-L23         312
SM-G950U Build/NRD90M               290
SM-G930V Build/NRD90M               274
rv:58.0                             269
rv:52.0                             256
SAMSUNG                             235
SM-G950F Build/NRD90M               225
Moto G (4) Build/NPJ25.93-14.7      219
SM-N950U Build/NMF26X               209
SM-A300H Build/LRX22G               203
hi6210sft Build/MRA58K              190
SM-J730GM Build/NRD90M              184


In [5]:
categoricalDomain=['id_12','id_15','id_16','id_23','id_27','id_28','id_29','id_30','id_31','id_33','id_34',
                   'id_35','id_36','id_37','id_38','DeviceType','DeviceInfo']
continuousDomain = []
for i in identity:
    if i not in categoricalDomain and i!='TransactionID' and i!='split':
        continuousDomain.append(i)

In [6]:
identity = identity.fillna(-1)
#step1 = ('label_encode', label_encoder_sk(cols=categoricalDomain)) 
step1 = ('onhot',OneHotClass(catego=categoricalDomain, miss='missing'))
step2 = ('MinMaxScaler', minmaxScalerClass(cols=continuousDomain,target="TransactionID"))

pipeline = Pipeline(steps=[step1,step2])
identity_new = pipeline.fit_transform(identity)

transfeature = [f for f in identity_new.columns if f!='TransactionID' and f!='split']
transfeature

['id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_13',
 'id_14',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_24',
 'id_25',
 'id_26',
 'id_32',
 'id_12_Found',
 'id_12_NotFound',
 'id_15_-1',
 'id_15_Found',
 'id_15_New',
 'id_15_Unknown',
 'id_16_-1',
 'id_16_Found',
 'id_16_NotFound',
 'id_23_-1',
 'id_23_IP_PROXY:ANONYMOUS',
 'id_23_IP_PROXY:HIDDEN',
 'id_23_IP_PROXY:TRANSPARENT',
 'id_27_-1',
 'id_27_Found',
 'id_27_NotFound',
 'id_28_-1',
 'id_28_Found',
 'id_28_New',
 'id_29_-1',
 'id_29_Found',
 'id_29_NotFound',
 'id_30_-1',
 'id_30_Android',
 'id_30_Android 4.4.2',
 'id_30_Android 5.0',
 'id_30_Android 5.0.2',
 'id_30_Android 5.1.1',
 'id_30_Android 6.0',
 'id_30_Android 6.0.1',
 'id_30_Android 7.0',
 'id_30_Android 7.1.1',
 'id_30_Android 7.1.2',
 'id_30_Android 8.0.0',
 'id_30_Android 8.1.0',
 'id_30_Android 9',
 'id_30_Linux',
 'id_30_Mac',
 'id_30_Mac OS X 10.10',
 'id_30_Mac OS X 10.11',

In [11]:
identity_new.to_csv('identity_new.csv',index=False)

In [10]:
encoding_dim=5
input_data = Input(shape=(3588,))
batch_size = 256
# encoder layers
encoded = Dense(2000, activation='relu')(input_data)
encoded = Dense(1000, activation='relu')(encoded)
encoded = Dense(500, activation='relu')(encoded)
encoded = Dense(100, activation='relu')(encoded)
encoded = Dense(50, activation='relu')(encoded)
encoder_output = Dense(encoding_dim,name='extract')(encoded)

# decoder layers
decoded = Dense(50, activation='relu')(encoder_output)
decoded = Dense(100, activation='relu')(decoded)
decoded = Dense(500, activation='relu')(decoded)
decoded = Dense(1000, activation='relu')(decoded)
decoded = Dense(2000, activation='tanh')(decoded)
decoded = Dense(3588, activation='tanh')(decoded)
# construct the autoencoder model
autoencoder = Model(input=input_data, output=decoded)
checkpoint = ModelCheckpoint('autoencoder_weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min')
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(identity_new[transfeature], identity_new[transfeature], epochs=20, batch_size =batch_size,verbose=0,validation_split=0.1,callbacks=[checkpoint])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


ValueError: Error when checking input: expected input_2 to have shape (40,) but got array with shape (3588,)

In [ ]:
def calf(row):
    input = [row[f] for f in feature]
    mean_squared_error([row[f] for f in feature], [autoencoder.predict(row[f]) for f in feature])

score = identity_new.apply(lambda row:calf(row),axis=1)

In [ ]:
identity_new['id_12']